## 下载数据集

In [2]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

/root/miniconda3/envs/LLM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [4]:
dataset["train"][123]

{'label': 2,
 'text': "Ah, Steak n Shake, you hit the spot after a night of drinking. Open 24 hours, drive thru or dine in, with wonderful, cheap greasy burgers, chicken fingers, breakfast sandwiches, and shoestring fries. You're faster (and cheaper!) than Eat N Park. Plus your employees wear those adorable bow ties. Granted, if we were at a fancy party, I might act like I don't know you, but deep down inside you know I heart you."}

In [5]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
    
show_random_elements(dataset["train"])

,label,text
0,5 stars,"I went onto Hotwire the other day and booked a hotel that is suppose to be only 3 stars. I didnt know the hotel but I had to keep it cheap and this place is what I got. I am so lucky! this place was amazing! The room was really clean, the staff is very helpful, the morning breakfast was pretty tasty and best of all it was really cheap in comparison to all of the other hotels in the area. It was at least $100 less then the ones in the same area. \nOh and there is a refrigerator and a microwave in the room!!"
1,4 stars,"Pretty good. Lots of different choices. I forget which day of the weekend they have seafood but, I loved the crab legs that you can get sauteed with a bunch of herbs and spices. All types of foods. The dessert section is great too. They have hand scooped ice cream or soft served and bunch of other selections."
2,1 star,"I was a customer for over 20 years until they charged me $158 for a $40 over draft. The problem is, it took them 10 days to send me a letter to inform me that I over drafted and then charged me every day. Considering that you have my email and phone number it would've been nice of you to inform me immediately that my account was overdrawn. Customer service is a joke and I had to call twice and leave a message plus email them to get a call back. Go somewhere else. Congrats PNC you lost yourself a customer."
3,5 stars,"Excellent Venezuelan food. Location is a Venezuelan style Arepera\""; the arepas are big, well stuffed and delicious. We ate the \""Hallacas\"" today, it's a Venezuelan type of tamal, and a traditional Christmas dish, it was really good. Passion fruit juice is an a must. Arepa of Queso de Mano, Reina Pepiada, Carne Mechada, are excellent! \nPlace is clean and has the \""Salto Angel\"" waterfall painted on the wall at the back of the restaurant, very nice! Service is friendly and excellent! This is our place to eat when we go to Charlotte!"
4,1 star,"What a horrible horrible place. They make you wait in line when nobody is inside the bar. Meanwhile, they let dozens of people in the front with no issue. They're trying to be LA or Vegas. Complete failure."
5,5 stars,"I am always an opened minded person and also can give the benefit of the doubt and also address credit when credit is due. So my last review about this place I was upset a little not because the food was bad or anything it was actually good but I am always getting teased about being hispanic and not able to even eat the mild sauce from taco bell without sweating and panting with my tongue for a drink. Lol\nBut I had ordered mild and received hot last time and wasn't pleased because I honestly at 4 wings and the rest sat in the fridge for a week before I threw them out. I had called and someone offered me 12 wings and I thought it was fair but I am not the type to really complain or go running to get free food like some people would do but needless to say I was contacted by vinnie and offered to replace my whole order and even a pizza. I offered to pay for the pizza because again I am not a free food person. I know nothing in this world is free unless it's your birthday then there is an exception. Ha ha ha\nWell I met with vinnie and he was awesome and very friendly and approachable I got the pizza and wings and able to eat the wings this time it was amazing. Best wings I have had in years they were juicy and crunchy and not small. I have been to some places where the wings are so small you get like one and a half bite of it but these wings are huge and the pizza that I was able to try for the first time was amazing as well. The cheese was flowing and piping hot and the sauce has to be a secret recipe past down from a family genius because it was perfect and not too over spiced or sweet. I am so glad he contacted me to try it out again and I know that our family Friday night pizza will be ordered from this place going forward in the future. My kids even ate the crust and not sure about other people's ki

## 预处理数据

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [7]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,3 stars,Not the best pet store. Prices not very cheap. It's very clean though. Will go back if only I'm in the area.,"[101, 1753, 1103, 1436, 11109, 2984, 119, 7510, 1116, 1136, 1304, 10928, 119, 1135, 112, 188, 1304, 4044, 1463, 119, 3100, 1301, 1171, 1191, 1178, 146, 112, 182, 1107, 1103, 1298, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


## 微调模型-加载 BERT 模型

In [8]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练超参数

In [9]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-yelp"

training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=32,
                                  num_train_epochs=5,
                                  logging_steps=500)

**查看完整超参数**

In [10]:
print(training_args)

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fs

## 训练过程中指标参数初始化

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=32,
                                  num_train_epochs=5,
                                  logging_steps=500)

# 开始训练
## 实例化训练器（Trainer）

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

## 训练

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.743300,0.737130,0.681740
2,0.684900,0.713017,0.689240
3,0.618300,0.715907,0.690760
4,0.502000,0.776169,0.687100


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter serve

**训练中 GPU内存占用情况**

```shell
Sat Mar 16 17:12:39 2024
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:CA:00.0 Off |                  Off |
| 36%   64C    P2             364W / 450W |  22547MiB / 24564MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
+---------------------------------------------------------------------------------------+
```


**用测试集再评估下试试看**

In [15]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.9002254605293274,
 'eval_accuracy': 0.68068,
 'eval_runtime': 245.1018,
 'eval_samples_per_second': 203.997,
 'eval_steps_per_second': 25.5,
 'epoch': 5.0}

## 保存模型和训练状态

In [16]:
trainer.save_model(model_dir)

In [17]:
trainer.save_state()